In [1]:
import pandas as pd
import requests
import requests
import json
import simplejson
import re
import boto3
from datetime import date
from datetime import datetime, timedelta
import subprocess
from datetime import datetime
import os
import pickle
# Gmail API utils
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
# for encoding/decoding messages in base64
from base64 import urlsafe_b64decode, urlsafe_b64encode
# for dealing with attachement MIME types
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from mimetypes import guess_type as guess_mime_type
import folium

In [2]:
SCOPES = ['https://mail.google.com/']
our_email = 'mjsmith9822@gmail.com'


In [3]:
def gmail_authenticate():
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('client_secret_mj.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)
    return build('gmail', 'v1', credentials=creds)

# get the Gmail API service
service = gmail_authenticate()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=550934624458-35lf9gj56b8l5ae5752o9i8e0fgknjj3.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62328%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=mnHJK4fqXPNPv8MoGsu6nWVHB7yrzY&access_type=offline


In [4]:
def add_attachment(message, filename):
    content_type, encoding = guess_mime_type(filename)
    if content_type is None or encoding is not None:
        content_type = 'application/octet-stream'
    main_type, sub_type = content_type.split('/', 1)
    print(main_type)
    if main_type == 'text':
        fp = open(filename, 'rb')
        msg = MIMEText(fp.read().decode(), _subtype=sub_type)
        fp.close()
    elif main_type == 'image':
        fp = open(filename, 'rb')
        msg = MIMEImage(fp.read(), _subtype=sub_type)
        fp.close()
    elif main_type == 'audio':
        fp = open(filename, 'rb')
        msg = MIMEAudio(fp.read(), _subtype=sub_type)
        fp.close()
    elif main_type == 'application':
        fp = open(filename, 'rb')
        msg = MIMEAudio(fp.read(), _subtype=sub_type)
        fp.close() 
    
    else:
        fp = open(filename, 'rb')
        msg = MIMEBase(main_type, sub_type)
        msg.set_payload(fp.read())
        fp.close()
    filename = os.path.basename(filename)
    msg.add_header('Content-Disposition', 'attachment', filename=filename)
    message.attach(msg)

In [5]:
def build_message(destination, obj, body, attachments=[]):
    if not attachments: # no attachments given
        message = MIMEText(body)
        message['to'] = destination
        message['from'] = our_email
        message['subject'] = obj
    else:
        message = MIMEMultipart()
        message['to'] = destination
        message['from'] = our_email
        message['subject'] = obj
        message.attach(MIMEText(body))
        for filename in attachments:
            add_attachment(message, filename)
    return {'raw': urlsafe_b64encode(message.as_bytes()).decode()}

In [6]:
def send_message(service, destination, obj, body, attachments=[]):
    return service.users().messages().send(
      userId="me",
      body=build_message(destination, obj, body, attachments)
    ).execute()

In [7]:
date=datetime.today().strftime('%Y-%m-%d')
nifc_url = "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/Current_WildlandFire_Locations/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json&fbclid=IwAR1rXNbPiRzmlAwn_jvLW1XHRD6aNCj2xLobwzc7n3pCcd_XZ9ZTiBA-YKQ"
req = requests.get(nifc_url)
req = req.json()
incident_df = pd.DataFrame(req['features'])

In [10]:
attribute_df = incident_df['attributes'].apply(pd.Series)
geometry_df = incident_df['geometry'].apply(pd.Series)
incident_complete_df = pd.concat([attribute_df,geometry_df],axis = 1)
incident_complete_df = incident_complete_df.rename(columns={'x':'Latitude', 'y':'Longitude'})
incident_complete_df['CreatedOnDateTime'] = pd.to_datetime(incident_complete_df['FireDiscoveryDateTime'], unit='ms')
incident_complete_df['TotalIncidentPersonnel'] = incident_complete_df['TotalIncidentPersonnel'].fillna(0)
incident_complete_df['Alert'] = incident_complete_df.apply(create_alert,axis=1)


incident_complete_df.to_csv('nifc_wildfires_' + date + '.csv')

In [9]:
def create_alert(incident_df):
    fire_name = incident_df['IncidentName']
    fire_size = incident_df['DailyAcres']
    num_firefighters = incident_df['TotalIncidentPersonnel']
    if fire_name != 'nan' and fire_size != 'nan' and num_firefighters != 'nan' and num_firefighters != 0:
        alert = fire_name + ' fire found at ' + str(fire_size) + ' acres, with ' + str(int(num_firefighters)) + ' firefighters in attendance.'
    elif fire_name and fire_size and not num_firefighters and fire_size != 'nan':
        alert = fire_name + ' fire found at ' + str(fire_size) + ' acres.'
    
    return alert    
    

In [11]:
fips_codes = pd.read_csv('US_fips_codes.csv')
fips_dict = fips_codes.set_index(str('fips')).T.to_dict()

In [ ]:
# try:
#     incident_complete_df['County'] = incident_complete_df.apply(lambda x: fips_dict.get(int(x['POOFips'])), axis=1)
# except:
#     incident_complete_df['County'] = 'No County Found'

In [12]:
send_message(service, 'kearns.dylan@gmail.com', 'Wildfires_'+date, str(incident_complete_df['Alert'][1]))   


{'id': '1837612a61e2d422',
 'threadId': '1837612a61e2d422',
 'labelIds': ['SENT']}

In [ ]:
incident_complete_df['Alert'][4]

In [ ]:
output = subprocess.check_output([r"C:\Program Files\IPFS Desktop\resources\app.asar.unpacked\node_modules\go-ipfs\go-ipfs\ipfs.exe","add",r"C:\Users\kearn\Desktop\Notebooks\nifc_wildfires_"+date+".csv"])
output = output.decode('utf-8')

In [ ]:
hashcode = output.split(' ')[1]
output_file = subprocess.check_output([r"C:\Program Files\IPFS Desktop\resources\app.asar.unpacked\node_modules\go-ipfs\go-ipfs\ipfs.exe","cat",hashcode])

In [ ]:
mymap = folium.Map()
for _,location in incident_complete_df.iterrows():

    folium.Marker(
    location=[location['Longitude'], location['Latitude']],
    popup=location['IncidentName']).add_to(mymap)
mymap

In [ ]:
mymap